In [1]:
import pandas as pd
import numpy as np
import folium
import os
from branca.element import Template, MacroElement

## this script is to generate an interactive map as an html file.
## Step1, execute a matlab script to extract the data from the MOSART output, and produce plots for each gauge location
## This step will also generate metrics for each gauge and put them into an .csv file
## Step2, execute this script to read the matlab generated .csv file, and link the interactive map to the gauge plots

# Function to map NSE values to colors
def edge_color(nse1, nse2):
    if nse1 < nse2:
        return 'red'
    elif nse1 > nse2:
        return 'blue'
    else:
        return '#000000'  # Black 

def nse_to_color(nse1, nse2, nse_diff_max = 1):
    nse_diff = nse1 - nse2
    if abs(nse_diff) >= nse_diff_max:
        return '#000000'  # Black for large difference
    elif nse_diff < 0:
        # Interpolate from red to white as NSE goes from -1 to 0
        red = 255
        green_blue = int(255 * (nse_diff + nse_diff_max) / nse_diff_max)
        return f'#{red:02x}{green_blue:02x}{green_blue:02x}'
    elif nse_diff == 0:
        return '#FFFFFF'  # White for NSE = 0
    else:
        # Interpolate from white to blue as NSE goes from 0 to 1
        blue = 255
        red_green = int(255 * (nse_diff_max - nse_diff) / nse_diff_max)
        return f'#{red_green:02x}{red_green:02x}{blue:02x}'

# Function to integrate plots with the map
def integrate_plots_with_map(df1, df2, casename1, casename2, base_path, plots_directory):
    # map_center = [df1['Lat (model)'].mean(), df1['Lon (model)'].mean()]
    map_center = [10, 0]
    map = folium.Map(location=map_center, zoom_start=3)

    # Set tile layer with no_wrap
    # folium.TileLayer('cartodbpositron').add_to(map)
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Terrain_Base/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri'
    ).add_to(map)
    
    # Adding Esri Hydro Reference Overlay
    esri_hydro = folium.TileLayer(
        tiles='https://tiles.arcgis.com/tiles/P3ePLMYs2RVChkJx/arcgis/rest/services/Esri_Hydro_Reference_Overlay/MapServer/tile/{z}/{y}/{x}',
        attr='Esri Hydro Reference',
        name='Esri Hydro Reference Overlay',
        overlay=True
    )
    esri_hydro.add_to(map)   
    
    # Determine the minimum size for a marker
    min_size = 1
    for index, row in df1.iterrows():
        gauge_id = row['ID']
        river_name = row['River']
        tooltip_text = f"River: {river_name}"
        plot1_path = os.path.join(base_path, casename1, plots_directory, f"{gauge_id}.png")
        plot2_path = os.path.join(base_path, casename2, plots_directory, f"{gauge_id}.png")
        
       # html = f'<img src="{plot1_path}" style="width:100%; height:auto;"> <img src="{plot2_path}" style="width:100%; height:auto;">'
       # iframe = folium.IFrame(html, width=300, height=600)  # Adjust width and height as needed
       # popup = folium.Popup(iframe, max_width=320)

        html = '''
        <div style="display: flex; justify-content: center; align-items: center;">
          <div>
            <div style="text-align: center;">{}</div>
            <img src="{}" style="width: 100%; height: auto; display: block;">
            </div>
            <div>
            <div style="text-align: center;">{}</div>
            <img src="{}" style="width: 100%; height: auto; display: block;">
          </div>
       </div>
       '''.format(casename1, plot1_path, casename2, plot2_path)

        iframe = folium.IFrame(html, width=600, height=350)  # Adjust width and height as needed
        popup = folium.Popup(iframe, max_width=620)

        nse1 = row['NSE']
        nse2 = df2.iloc[index]['NSE'] 
        color = nse_to_color(nse1, nse2)
        edgecolor = edge_color(nse1, nse2)

        # Calculate marker size using logarithmic scale
        # Add a small value to avoid log(0) and log(negative) issues
        size = np.log(row['Annual meanQ obs'] + 1) * min_size * 1.5
        
        folium.CircleMarker(
            location=[row['Lat (model)'], row['Lon (model)']],
            popup=popup,
            tooltip=tooltip_text,
            radius = size,
            color=edgecolor,
            weight=1,
            fill=True,
            fill_color=color,
            fill_opacity=0.7 
        ).add_to(map)
    
    return map

# Base path and plots directory

dirname = '2024_Tutorial'
casename1 = 'GRFR'
casename2 = 'extendedOutput.v3.LR.historical_0101'
www_root = "https://portal.nersc.gov/cfs/e3sm/tizhou/"
www_path = www_root + dirname + "/"
diag_root = "P:\\global\\cfs\\cdirs\\e3sm\\www\\tizhou\\"
plots_directory = "gauge_plots"

# CSV file path under the case path
csv_file_path1 = os.path.join(diag_root, dirname, casename1, "gauge_data.csv")
csv_file_path2 = os.path.join(diag_root, dirname, casename2, "gauge_data.csv")

# Read the gauge data from CSV
df1 = pd.read_csv(csv_file_path1)
df2 = pd.read_csv(csv_file_path2)

max_abs_diff = max(df1['NSE'] - df2['NSE'].abs().max())

# Generate the map
my_map = integrate_plots_with_map(df1, df2, casename1, casename2, www_path, plots_directory)

# HTML and CSS template for the legend
legend_html = '''
{{% macro html(this, kwargs) %}}
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 500px; height: 100px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color: white; padding: 5px;">
    <div><strong>Legend (NSE diff)</strong></div>
    <div style="width: 480px; height: 20px; 
                background: linear-gradient(to right, red, white, blue);"></div>
    <div style="display: flex; justify-content: space-between; 
                width: 480px;">
        <span>{} is better</span><span>{} is better</span>
    </div>
    <div style="margin-top: 5px;">Size of the dot reflects mean annual discharge</div>
</div>
{{% endmacro %}}
'''.format(casename2, casename1)

# Create a MacroElement with this template
macro = MacroElement()
macro._template = Template(legend_html)

# Add to your existing Folium map
my_map.get_root().add_child(macro)

# Save or show the map
output_dir = os.path.join(diag_root, dirname, 'comparisons')
os.makedirs(output_dir, exist_ok=True)

output_file = os.path.join(output_dir, f'{casename1}_vs_{casename2}.html')
my_map.save(output_file)